Preparing the data

In [85]:
# !pip install autograd
import autograd.numpy as np
import pandas as pd
from scipy.stats import norm

data = pd.read_csv("data425288.csv")

T = 25
N = data.ID.max()

y = np.log(data.Sales.values.reshape((N,T)))

X = np.ones((T,2))
X[:,1] = np.log(data.Price[:T])

OLS for reference

In [77]:
import statsmodels.formula.api as sm

regdata = data.copy()
regdata["logS"] = np.log(regdata.Sales)
regdata["logP"] = np.log(regdata.Price)
result = sm.ols(formula="logS ~ logP", data=regdata).fit()
display(result.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   logS   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     4408.
Date:                Mon, 09 Dec 2019   Prob (F-statistic):               0.00
Time:                        20:56:04   Log-Likelihood:                -19197.
No. Observations:               12500   AIC:                         3.840e+04
Df Residuals:                   12498   BIC:                         3.841e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.7127      0.023    462.944      0.000      10.667      10.758
logP          -1.3974      0.021    -66.393      0.000      -1.439      -1.356
==============================================================================
Omnibus:                        1.098   Durbin-Watson:                   1.596
Prob(Omnibus):                  0.577   Jarque-Bera (JB):                1.113
Skew:                          -0.002   Prob(JB):                        0.573
Kurtosis:                       2.954   Cond. No.                         4.40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Log-likelihood evaluation

In [78]:
def LogL(theta, pi, y, X):
    K=theta.shape[0]
    mu = np.dot(X,theta.T) #(25, 3)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,K)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,K)
    
    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,K)
    
    #prod^T probs
    segments = np.prod(probs, axis=1) #(500,K)
    
    #sum^K pi_c prod^T probs
    combined = np.dot(segments, pi) #(500,)
    
    #sum^N log sum^K pi_c prod^T probs
    LogL = np.log(combined).sum() #(1,)
    return LogL

Expectation step

In [79]:
def EStep(theta,pi,y,X, robust = True):
    K=theta.shape[0] 
    mu = np.dot(X,theta.T) #(25, K)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,K)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,K)

    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,K)
    
    #if to be calcluted by exp(log()) to prevent underflow
    if robust:
        #prod^T probs as sum^T log p_t to prevent small numbers
        segments = np.log(probs).sum(axis=1)
        
        # prod^T probs times diagonal of pi as exp(sum^T log p_t + log pi_c - most negative number)
        numerators = np.exp(segments + np.log(pi) - segments.min())    
    
    #using the direct definition
    else:
        #prod^T probs
        segments = np.prod(probs, axis=1) #(500,K)
        
        # prod^T probs times diagonal of pi
        numerators = np.dot(segments, np.diag(pi))
    
    #divide numerators by denominators (= sum of row)
    W = numerators / numerators.sum(axis=1, keepdims=True)

    return W

Maximization step

In [80]:
def MStep(W,y,X):
    K=W.shape[1]
    
    # 1/N sum^N w_ic
    pi = W.mean(axis=0) #(K,)
    
    theta = np.zeros((K,2))
    
    for c in range(K):        
        inv = np.zeros((2,2))
        for i in range(N): 
            inv += W[i,c] * np.dot(X.T, X) 
        inv  =  np.linalg.inv(inv)
        
        theta[c,:] = np.linalg.multi_dot([inv,X.T,y.T, W[:,c]])
    
    return theta, pi

EM algorithm

In [81]:
def EM(K,y,X,tolerance=0.0001, verbose=0):
    #W initialized randomly
    W = np.random.rand(N,K)
    W = W/W.sum(axis=1, keepdims=True)
    theta = None
    pi = None
    iterations=0
    max_iter = 250
    
    diff = np.inf
    while (diff >= tolerance and iterations < max_iter):
        iterations+=1
        theta, pi = MStep(W,y,X)
        currentW = EStep(theta,pi,y,X, robust = True)
        diff = np.linalg.norm(W-currentW)
        W = currentW
        
        if verbose==2:            
            print("Iteration "+str(iterations))
            print("Log-likelihood: "+str(LogL(theta, pi, y, X)))
            print("\n")
    
    if(iterations == max_iter): print("Maximum iterations reached, iters: ",max_iter)
    if verbose >= 1:
        print("Log-likelihood: "+str(LogL(theta, pi, y, X)))
    
    return theta, pi

Estimation implementation

In [93]:
from autograd import hessian 

def Estimate(K, X=X, y=y, tolerance=0.1, seed=1234,verbose=2, n_est=10):
    theta = None
    pi = None
    Likelihood = -np.inf
    np.random.seed(seed)
    
    for i in range(n_est):
        if verbose >= 1: print("\nEstimation ",i+1)
            
        t, p = EM(K,y,X,tolerance=tolerance, verbose=verbose)
        currentLogL = LogL(t, p, y, X)
            
        if currentLogL >= Likelihood:
            theta = t
            pi = p
            Likelihood = currentLogL
    
    if verbose >= 1:
        print("The estimation is finished")
        print("\nLog likelihood: "+str(Likelihood))
        print("\nTheta:")
        print(theta)
        print("\nPi:")
        print(pi)
    
    gamma = np.zeros(K)
    for c in range(K-1):
        gamma[c] = np.log(pi[c]) - np.log(pi[K-1])
    
    #Restricted Log Likelihood implementation
    def RLogL(theta):
        pi = np.exp(gamma) / np.exp(gamma).sum()
        return LogL(theta, pi, y, X)
    
    print(hessian(RLogL)(theta))
#     hessian = 
    
        
# return theta, pi, Likelihood

Estimate(K=2, X=X, y=y, tolerance=0.001, verbose=0, n_est=1)

ValueError: setting an array element with a sequence.